<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/AE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle #必要的
training_data = pickle.load(open("training_data.pkl","rb")) #必要的
testing_data = pickle.load(open("testing_data.pkl","rb")) #必要的

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data))) #8000
print("Size of testing dataset: {0}".format(len(testing_data))) #2000
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")
#LABEL: neg / SENTENCE: hopeless for tmr :(

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


In [2]:
 import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#先搞个label。再搞个sentence
#train_data = [training_data[i][1] for i in range(len(training_data))]

##print(train_data[0]) #hopeless for tmr :(
##test:'@AndreaMarySmith very helpful .... Or will be once I stop crying :('
'''
train_data =[]
for i in range(len(training_data)):
  train_data.append(training_data[i][1])
print(train_data[0]) 
#上下一样
'''

train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]

test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos


In [4]:
#做预处理 变小写
import pprint
text_train = [s.lower() for s in train_data]
text_test = [s.lower() for s in test_data]


In [5]:

import pprint
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()


train_t=[]  #完全每句话的分词的集合。train_t[0][0]每个单词
for s in text_train:
  text_train = tknzr.tokenize(s)                         #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
pprint.pprint(train_t[:5])         #:(    :-(


test_t=[]     #2000
for w in text_test:
  text_test = tknzr.tokenize(w)
  test_t.append(text_test)
pprint.pprint(text_test[:5])


[['hopeless', 'for', 'tmr', ':('],
 ['everything',
  'in',
  'the',
  'kids',
  'section',
  'of',
  'ikea',
  'is',
  'so',
  'cute',
  '.',
  'shame',
  "i'm",
  'nearly',
  '19',
  'in',
  '2',
  'months',
  ':('],
 ['@hegelbon',
  'that',
  'heart',
  'sliding',
  'into',
  'the',
  'waste',
  'basket',
  '.',
  ':('],
 ['“',
  '@ketchburning',
  ':',
  'i',
  'hate',
  'japanese',
  'call',
  'him',
  '"',
  'bani',
  '"',
  ':(',
  ':(',
  '”',
  'me',
  'too'],
 ['dang', 'starting', 'next', 'week', 'i', 'have', '"', 'work', '"', ':(']]
['@danieloconnel18', 'you', 'could', 'say', 'he']


In [6]:
#theprincesszooz but i see what youre going at   
#yes  subjective pain may not be real
#  but that does not make it less painful

import re
def remove(x):
  t = []
  for i in range(len(x)):
    t_sub = []        #是直接用空列表代替了
    for j in range(len(x[i])):
      if len(x[i][j])==0:
        continue
      if x[i][j] == " ":
        continue 
      else:
        x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
        if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
          x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
        if len(x[i][j]) == 0:
          continue 
        else:
          t_sub.append(x[i][j]) 
    t.append(t_sub) 

  return t

text_train = remove(train_t)
text_test = remove(test_t)
print(text_train[:5])
#print('==')
#pprint.pprint(test_t[:5])

[['hopeless', 'for', 'tmr', ':('], ['everything', 'in', 'the', 'kids', 'section', 'of', 'ikea', 'is', 'so', 'cute', 'shame', "i'm", 'nearly', 'in', 'months', ':('], ['hegelbon', 'that', 'heart', 'sliding', 'into', 'the', 'waste', 'basket', ':('], ['ketchburning', 'i', 'hate', 'japanese', 'call', 'him', 'bani', ':(', ':(', 'me', 'too'], ['dang', 'starting', 'next', 'week', 'i', 'have', 'work', ':(']]


In [7]:
#停用词  
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()
sww = sw.words()

text_train_ns=[]    #8000
for tokens in text_train:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_ns.append(filtered_sentence1)
pprint.pprint(text_train_ns[:5])

text_test_ns=[]   #2000
for tokens in text_test:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_ns.append(filtered_sentence2)
pprint.pprint(text_test_ns[:5])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[['hopeless', 'tmr', ':('],
 ['everything',
  'kids',
  'section',
  'ikea',
  'cute',
  'shame',
  "i'm",
  'nearly',
  'months',
  ':('],
 ['hegelbon', 'heart', 'sliding', 'waste', 'basket', ':('],
 ['ketchburning', 'hate', 'japanese', 'call', 'bani', ':(', ':('],
 ['dang', 'starting', 'next', 'week', 'work', ':(']]
[['andreamarysmith', 'helpful', '...', 'stop', 'crying', ':('],
 ['realyys_', 'otl', 'nevermind', ':(', 'least', 'got', 'jeon'],
 ['soon',
  'tweeted',
  'planted',
  'claws',
  'thigh',
  'traction',
  'zoomed',
  'away',
  ':('],
 ['luketothestars', 'damnit', ':('],
 ['klm',
  'used',
  'pry',
  'pv',
  '...',
  'wish',
  'could',
  'relive',
  'days',
  'become',
  'nyc',
  'pv',
  'buy',
  'way',
  'communicate',
  'nyc',
  'usa',
  'klm',
  ':(']]


In [8]:
#找词根
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text_train_le = []
for tokens in text_train_ns:
  lemma_sentence1 = [lemmatizer.lemmatize(w) for w in tokens ]
  text_train_le.append(lemma_sentence1)
pprint.pprint(text_train_le[:10])

text_test_le = []
for tokens in text_test_ns:
  lemma_sentence2 = [lemmatizer.lemmatize(w) for w in tokens]
  text_test_le.append(lemma_sentence2)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['hopeless', 'tmr', ':('],
 ['everything',
  'kid',
  'section',
  'ikea',
  'cute',
  'shame',
  "i'm",
  'nearly',
  'month',
  ':('],
 ['hegelbon', 'heart', 'sliding', 'waste', 'basket', ':('],
 ['ketchburning', 'hate', 'japanese', 'call', 'ban', ':(', ':('],
 ['dang', 'starting', 'next', 'week', 'work', ':('],
 ['oh', 'god', 'baby', 'face', ':(', 'httpstcofcwgvaki'],
 ['rileymcdonough', 'make', 'smile', ':('],
 ['fggstar',
  'stuartthull',
  'work',
  'neighbour',
  'motor',
  'asked',
  'said',
  'hate',
  'update',
  'search',
  ':(',
  'httptcoxvmtuikwln'],
 [':(', 'tahuodyy', 'sialan', ':(', 'httpstcohvixcrl'],
 ['athabasca',
  'glacier',
  ':-(',
  'athabasca',
  'glacier',
  'jasper',
  'jaspernationalpark',
  'alberta',
  'explorealberta',
  'httptcodzzdqmfcz']]


In [9]:
#去掉复数 stemmer 词根不需要这部； 如果用就需要在neg和pos需要stemmer操作
from nltk.stem.porter import *
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
for i in range(len(text_train_le)):
  for plural in text_train_le[i]:
    singles = stemmer.stem(plural)
print(text_train_le[:5])

#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
for i in range(len(text_test_le)):
  for plural in text_test_le[i]:
    singles = stemmer.stem(plural)
print(text_test_le[:5])


[['hopeless', 'tmr', ':('], ['everything', 'kid', 'section', 'ikea', 'cute', 'shame', "i'm", 'nearly', 'month', ':('], ['hegelbon', 'heart', 'sliding', 'waste', 'basket', ':('], ['ketchburning', 'hate', 'japanese', 'call', 'ban', ':(', ':('], ['dang', 'starting', 'next', 'week', 'work', ':(']]
[['andreamarysmith', 'helpful', '...', 'stop', 'cry', ':('], ['realyys_', 'otl', 'nevermind', ':(', 'least', 'got', 'jeon'], ['soon', 'tweeted', 'planted', 'claw', 'thigh', 'traction', 'zoomed', 'away', ':('], ['luketothestars', 'damnit', ':('], ['klm', 'used', 'pry', 'pv', '...', 'wish', 'could', 'relive', 'day', 'become', 'nyc', 'pv', 'buy', 'way', 'communicate', 'nyc', 'usa', 'klm', ':(']]


#WORD embedding

In [ ]:
#https://colab.research.google.com/drive/112veBlTX6pVhQWCexXbc_BLgue_lb_Ox#scrollTo=2FKp3X7pkRm6
#好像解决的是评估 貌似是glove

**2.1** .g. word2vec-CBOW, word2vec-Skip gram, fasttext, glove) can be applied. (Justify your decision)

In [109]:
# Preprocess data for word embeddings: 1中的数据集做词嵌入。 那这个就已经可以了就用上面那个
#Build training model for word embeddings:  word2vec-CBOW, word2vec-Skip gram, fasttext, glove)
#(dimension of embeddings, window size, learning rate, etc.)
#train model:  这个就是创建模型
from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences= text_train_le + text_test_le, size=100, window=5, min_count=2, workers=2, sg=0)
print(wv_cbow_model.most_similar(":("))
#wv_cbow_model = Word2Vec(sentences= text_test_le, size=100, window=5, min_count=1, workers=2, sg=0)
#print(wv_cbow_model.wv.most_similar(":("))
#print(wv_cbow_model.wv.most_similar(":(") == wv_cbow_model.most_similar(":("))
#这里的词有些不在有些不在  tmr
#https://blog.csdn.net/qq_32505207/article/details/106456260?ops_request_misc=&request_id=&biz_id=102&utm_term=KeyError:%20%22word%20%27tmr%27%20not%20in%20v&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-3-106456260.first_rank_v2_pc_rank_v29


[('check', 0.9999226331710815), ('weekend', 0.9999089241027832), ('friend', 0.9999086260795593), ('dm', 0.9999076724052429), ('hey', 0.9999074935913086), ('always', 0.9999011158943176), ('give', 0.9999002814292908), ('pls', 0.9999001026153564), ('thank', 0.9998989105224609), ('best', 0.9998985528945923)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


2.2 Lexicon Embedding

In [ ]:
#f = open('your_file.txt', 'r', encoding='utf-8') #或者io，使用哪种包无所谓
with open('your_file.txt', 'r', 'utf-8') as f: f.write('This method is prior')

In [142]:
'''挂载你云盘的操作
from google.colab import drive
drive.mount('/content/drive')
'''
with open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding='utf-8') as f:
  negative_words = np.copy(f)

In [ ]:
#否定词"ISO-8859-1"==UTF-8
not_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件呢。
j = -1
for i in not_words:
  j +=1
  if i == '2-faced':
    break

print(j)  #31
print(not_words[j])  #2-faced
#把背景介绍删除
neg = not_words[j:]
print(len(neg)) #4783
print(neg[0]) #2-faced
print(neg[:10])
#肯定"ISO-8859-1"==UTF-8
sure_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件呢。
j = -1
for i in sure_words:
  j +=1
  if i == 'a+':
    break

print(j)  #31
print(sure_words[j])  #2-faced
#把背景介绍删除
pos = sure_words[j:]
print(len(pos)) #2006
print(pos[0]) #a+
print(pos[:10])

#neg pos 词表

In [ ]:
stop

In [ ]:
#【不要】学习加载文件
'''
with open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding='utf-8') as f: f.write('This method is prior')
#/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt
#/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt
#/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English

'''
'''
#加载数据，要知道文件夹在哪里，导入数据集
#显示数据集里有什么
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/train/images_training.h5", "r") as f:
  data_train = np.copy(f["datatrain"])
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/train/labels_training.h5", "r") as f:
  label_train = np.copy(f["labeltrain"])
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/test/images_testing.h5", "r") as f:
  data_test = np.copy(f["datatest"])  
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/test/labels_testing_2000.h5", "r") as f:
  label_test = np.copy(f["labeltest"])
  '''

#train

In [111]:
wv_cbow_model.save("cbow.model")
#训练 ； 存 取；训练。
cbow = Word2Vec.load("./cbow.model")
cbow.train(sentences= text_train_le + text_test_le ,total_examples=wv_cbow_model.corpus_count, epochs=cbow.iter)
#(235832, 368710)
#https://blog.csdn.net/yibo492387/article/details/81908249?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161909968516780261919053%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161909968516780261919053&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-81908249.first_rank_v2_pc_rank_v29&utm_term=Word2vec.train
#model.train([["hopeless", "weekend"]]), total_examples=1, epochs=1)  # #基于新语料更新词向量表示
#https://blog.csdn.net/baidu_26550817/article/details/48653889?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161910098016780262535664%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161910098016780262535664&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~baidu_landing_v2~default-1-48653889.first_rank_v2_pc_rank_v29&utm_term=word2vec.train参数
#【上面是解释训练的这些参数是什么】
#https://blog.csdn.net/qq_19707521/article/details/79168239?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161910098016780262535664%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161910098016780262535664&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~baidu_landing_v2~default-2-79168239.first_rank_v2_pc_rank_v29&utm_term=word2vec.train参数
#https://blog.csdn.net/UZDW_/article/details/114374822?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161910098016780261927889%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161910098016780261927889&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-7-114374822.first_rank_v2_pc_rank_v29&utm_term=word2vec.train参数


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


(235832, 368710)

In [114]:
vector = cbow.wv['check']
vector

array([ 0.24533346,  0.44094914, -0.4012232 ,  0.42925003, -0.15039185,
       -0.18189919, -0.3698725 , -0.31360888, -0.21066356,  0.0024024 ,
        0.140888  , -0.37234405, -0.26162758, -0.24837701, -0.29326162,
        0.23988765, -0.09268534,  0.09694377, -0.04807883,  0.0172402 ,
        0.57437056,  0.05977219, -0.21236305, -0.323197  , -0.71285313,
        0.31709996, -0.00889755, -0.19448394,  0.5780465 , -0.06199858,
       -0.06583932,  0.6789193 , -0.0884474 ,  0.11800878,  0.24761231,
       -0.31728932, -0.3344343 , -0.1697669 ,  0.0279294 ,  0.1186721 ,
        0.4874106 , -0.5675884 ,  0.08422088, -0.289564  , -0.5550803 ,
        0.13189134, -0.37766716, -0.11126556,  0.1416423 ,  0.9374534 ,
        0.5736548 ,  0.0803565 ,  0.35511765,  0.08001357,  0.256081  ,
       -0.45159653, -0.08400764,  0.392848  , -0.47912532, -0.07102933,
       -0.12574245, -0.09913579,  0.27236155, -0.25244164, -0.48290366,
       -0.5977871 ,  0.3234435 ,  0.11617676, -0.63462806,  0.47

In [ ]:
#给出的训练
'''
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model2 = Word2Vec.load("./word2vec.model")
model2.train(common_texts,total_examples=model.corpus_count, epochs=model2.iter)
 '''
'''
from gensim.models import Word2Vec as wtv
word_model = wtv.load("C:/model/word2vec_model")  # 加载模型词向量参数
word_model.train([["我","爱","你","们"]], total_examples=1, epochs=10)  #基于新语料更新词向量表示
【不断加载模型】

 这种方法可以对现有word2vec的词向量进行更新，在实操的时候，train函数中的训练数据如果出现新的词，并不会更新到现有的词向量矩阵中。

举个例子，已经训练好的word2vec词表有[“我”,“爱”,“你”]三个词，给定一个新语料[“我”,“喜”,“欢”,“你”]，新语料只会调整原有词中的“我”“你”的词向量表示，而不会将“喜”“欢”添加到词库中。
#https://blog.csdn.net/qq_37388085/article/details/108289649?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161909860016780271575634%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161909860016780271575634&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-11-108289649.first_rank_v2_pc_rank_v29&utm_term=如何直接训练Word2Vec
'''

**2.2 Lexicon Embedding**

In [ ]:
'''
#读取否定词文件
not_word_file = open('否定词.txt','r+',encoding='utf-8')
not_word_list = not_word_file.readlines()
degree_dict = defaultdict()
'''

In [ ]:
word_set = set() 
for sent in text_train_le:#获取单词列表的输出样本text_train_le    text_test_le
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index) #word_index
#这里需要的是word_list

In [87]:
# The sequence length is pre-defined, you can't change this value for this exercise
#应该是seq2seq
seq_length = 100   #回头要弄 [得弄个for循环看看最长句子要多少]

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 
print(train_pad_encoded[:2])

[[5836, 14705, 38, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72], [4216, 8488, 12868, 7434, 3084, 13033, 7339, 10434, 10061, 38, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72]]


In [ ]:
# Get the Embedding lookup table
import numpy as np
emb_dim = wv_cbow_model.vector_size

emb_table = []
for i, word in enumerate(word_list):
    if word in wv_cbow_model:
        emb_table.append(wv_cbow_model[word])
    else:
        emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)  #目前是个array

In [121]:
print(emb_table.shape) #(16283, 100)

(16283, 100)


In [ ]:
lookup_tensor = torch.tensor(emb_table,dtype=torch.long) #是不是全是0啊？？？？把array变tensor

In [122]:
lookup_tensor.shape #torch.Size([16283, 100])

torch.Size([16283, 100])

In [98]:
#lab3 所以训练要用nn.model
voc_size = len(word_list)
learning_rate = 0.1
batch_size = 16
embedding_size = 2
no_of_epochs = 5000

test

In [89]:
emb_table

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03059139,  0.08198201, -0.09884707, ..., -0.14043565,
        -0.07070709, -0.01905875],
       [ 0.02154521,  0.04067583, -0.05620475, ..., -0.0742216 ,
        -0.04326894, -0.0160607 ],
       ...,
       [ 0.01317871,  0.02044111, -0.02249284, ..., -0.02864111,
        -0.01433067, -0.00431645],
       [ 0.08975952,  0.12070724, -0.13421866, ..., -0.20293678,
        -0.0964682 , -0.02566609],
       [ 0.11756032,  0.14386623, -0.16996379, ..., -0.25028536,
        -0.12735364, -0.04037781]])

In [84]:
lookup_tensor = torch.tensor(emb_table,dtype=torch.long) #是不是全是0啊？？？？

In [80]:
emb_table #16283,100  #这个是个100
emb_table[1] #
lookup_tensor[2]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [ ]:
wv_cbow_model.wv['hopeless']

**Get Word List**

In [20]:
word_set = set() 
for sent in text_train_le:#获取单词列表的输出样本text_train_le    text_test_le
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index) #word_index

['\n\n', ' ', '  ', '#am', '#bb', '#bc', '#be', '#cc', '#eh', '#ff', '#fm', '#hr', '#ku', '#ll', '#me', '#rp', '#t', '#ti', '#vr', '#عن', '#더쇼', '(-:', '(:', '):', '+', '--', '-->', '->', '.\n.', '. .', '.+', '.-', '..', '...', '/:', ":'(", ":')", ":'d", ':(', ':)', ':-(', ':-)', ':-d', ':/', '::', ':\\', ':d', ':p', ':|', ';(', ';)', ';-)', ';p', ';}', '=:', '=d', '>:(', '>:)', '>:d', '@_', '@am', '@ar', '@ee', '@ew', '@fm', '@ft', '@gh', '@o', '@oc', '@pm', '@rx', '@sp', '[PAD]', '[UNKNOWN]', '_', '__', '___', '__crow__', '__final_final_final', '__giveawaydms_', '__karabom', '__riixc', '__sabaa', '_abbeytaylor', '_ainzuhraa', '_ajaaaa', '_andree', '_angelina_horan', '_atirahmatasri', '_avonsparadise_', '_awkwardraven', '_ayeayecaptain', '_beatrizhoran_', '_bibboo', '_birexus', '_bitsenpai_', '_blacktomorrow', '_bleh', '_bonakid', '_bushraaaaaa', '_caitlintierney', '_cigarella', '_cruuzzy', '_dedox', '_donutbethers_', '_dreyaniks', '_edmce', '_emceeeee', '_ennyselise', '_escn', '_esha

**padding and encoding [Please complete this section]**

In [21]:
# The sequence length is pre-defined, you can't change this value for this exercise
seq_length = 16   #回头要弄 [得弄个for循环看看最长句子要多少]

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 

In [57]:
train_pad_encoded[0]

[5836, 14705, 38, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72]

2)Lexicon Embedding (2 marks) 012 还要处理OVV

#TEST

In [23]:
len(train_pad_encoded) # [5549, 7921, 2087, 1222, 38, 38, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72],

8000

In [18]:
wv_cbow_model.vector_size

100

#**Model**

In [11]:
'''
1 要先准备数据； 做分词预处理的行为。
差的是构建模型。
2 要构建训练模型： wword（………………………………）
只是创建完，回头数据还是要改。但明确了哪里是哪里很重要。
'''

# Initialize and train a word2vec model with the following parameters:
# sentence: iterable of iterables, i.e. the list of lists of tokens from our data
# size: dimensionality of the word vectors
# window: window size
# min_count: ignores all words with total frequency lower than the specified count value
# workers: Use specified number of worker threads to train the model (=faster training with multicore machines)
# sg: training algorithm, 0 for CBOW, 1 for skip-gram
from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(wv_cbow_model.most_similar(":("))
wv_cbow_model = Word2Vec(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(wv_cbow_model.wv.most_similar(":("))
print(wv_cbow_model.wv.most_similar(":(") == wv_cbow_model.most_similar(":("))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


[('pls', 0.9997409582138062), ('friend', 0.9997279644012451), ('always', 0.9997177124023438), ('check', 0.9996904134750366), ('dm', 0.999685525894165), ('baby', 0.9996812343597412), ('life', 0.9996784925460815), ('hey', 0.9996722936630249), ('blog', 0.9996721148490906), ('help', 0.9996641874313354)]
[(':)', 0.9994966387748718), (':-)', 0.9990910291671753), ("i'm", 0.9989172220230103), ('day', 0.9988608360290527), ('get', 0.9987145662307739), ('love', 0.9986813068389893), ('time', 0.9986474514007568), ('please', 0.998597264289856), ('think', 0.9985897541046143), (':-(', 0.998504638671875)]
True


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [12]:
#word2vec skip gram
# Now we switch to a Skip Gram model by setting parameter sg=1
wv_sg_model = Word2Vec(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=1)
print(wv_sg_model.most_similar(":("))
wv_sg_model = Word2Vec(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=1)

print(wv_sg_model.wv.most_similar(":("))
print(wv_sg_model.wv.most_similar(":(") == wv_sg_model.most_similar(":("))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[('working', 0.9861922860145569), ('sharing', 0.9844731688499451), ('httptcouzoaqrowkx', 0.9828289747238159), ('following', 0.9810550212860107), ('blog', 0.980736255645752), ('okay', 0.9798705577850342), ('pls', 0.9798064231872559), ('ignored', 0.979103684425354), ('wishing', 0.9788521528244019), ('pick', 0.9781317710876465)]
[(':)', 0.9997023940086365), (':-)', 0.9996683597564697), ('people', 0.999656081199646), ('guy', 0.9996546506881714), ('dont', 0.9996299743652344), ('love', 0.999627947807312), ('get', 0.9996195435523987), (':-(', 0.9996161460876465), ('really', 0.9996123313903809), ('year', 0.9996036887168884)]
True


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [13]:
#FastText -skip gram
from gensim.models import FastText
# Now we initialize and train FastText with Skip Gram architecture (sg=1)
ft_sg_model = FastText(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=1)
print(ft_sg_model.most_similar(":("))
ft_sg_model = FastText(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=1)
print(ft_sg_model.wv.most_similar(":("))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


[('appreciate', 0.9974219799041748), ('clean', 0.9973695278167725), ('badly', 0.9970263838768005), ('hand', 0.9970142245292664), ('must', 0.9970107078552246), ('ball', 0.996971845626831), ('youve', 0.9969593286514282), ('mum', 0.996914267539978), ('jungsilhoon', 0.9968662858009338), ('plz', 0.9968521595001221)]
[('unfortunately', 0.9997067451477051), ('start', 0.9996899366378784), ('people', 0.9996864795684814), ('lovely', 0.99968421459198), ('sunday', 0.9996838569641113), ('read', 0.9996805191040039), ('london', 0.9996793270111084), ('work', 0.9996756315231323), ('service', 0.9996744394302368), ('found', 0.9996738433837891)]


In [14]:
#FastText -cbow
# Now we initialize and train FastText with CBOW architecture (sg=0)
ft_cbow_model = FastText(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(ft_cbow_model.most_similar(":("))
ft_cbow_model = FastText(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(ft_cbow_model.most_similar(":("))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[('interesting', 0.9999589323997498), ('finger', 0.9999535083770752), ('couple', 0.9999514818191528), ('play', 0.9999512434005737), ('complete', 0.9999511241912842), ('course', 0.9999511241912842), ('another', 0.9999502897262573), ('event', 0.9999502897262573), ('otherwise', 0.9999491572380066), ('interested', 0.9999489188194275)]
[('day', 0.9998699426651001), ('follower', 0.9998611807823181), ('thing', 0.9998561143875122), ('follow', 0.999854326248169), ('everything', 0.9998528957366943), ('going', 0.9998511672019958), ('following', 0.9998509883880615), ('watching', 0.9998447895050049), ('everyone', 0.999843955039978), ('looking', 0.9998409748077393)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


#Lexicon embeddings 不是这个是要输入句子。把句子加上word+embedding

**Label encodding[标签编码]**

In [15]:
import numpy as np

from sklearn.preprocessing import LabelEncoder

unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
# Please encode the labels (Do NOT add new lines of code in this section)
# Hint: Try to understand the difference between fit_transform and transform
#To normalise the data (so that the variance of the feature data is 1 and the mean is 0), the mean μ and variance σ^2 of the feature data are calculated. fit is not used because the training set (fit) has already found the mean and variance. (The transformation rule is found), and this rule is used directly.
label_train_encoded = lEnc.fit_transform(train_label)
label_test_encoded = lEnc.transform(test_label)

n_class = len(unique_labels)
print(unique_labels)
print(lEnc.transform(unique_labels))
#not_exist(0), negative(1), and positive(2)


['neg' 'pos']
[0 1]


**get word list**

In [16]:
word_index['[PAD]']   #72 [] 不是（）
word_index['[UNKNOWN]']# 73

NameError: ignored

**padding and encoding**

In [ ]:
# The sequence length is pre-defined, you can't change this value for this exercise
seq_length = 16

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 

*[可选]您需要说明为什么您想使用超过一维的嵌入。
*[Optional] You are required to describe why you would like to use more than one-dimensional embedding.*

In [ ]:
wv_cbow_model.vector_size

In [ ]:
word_set = set() 
for sent in text_train_le:
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index)

In [ ]:
#创建嵌入查询表  embedding lookup table 【是传说中的W吗？】 【为什么要嵌入这个词表】

import numpy as np
emb_dim = wv_cbow_model.vector_size    #要训练的数据集的向量大小
emb_table = []
for i, word in enumerate(word_list):  #['[PAD]', '[UNKNOWN]', 'do', 'hate', 'i', 'it', 'like', 'love', 'not', 'that']
  if word in wv_cbow_model:
    emb_table.append(wv_cbow_model[word])
  else:
    emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)  #16283

In [ ]:
emb_table.shape#(16283, 100)

In [ ]:
#创建嵌入查询表  embedding lookup table 【是传说中的W吗？】 【为什么要嵌入这个词表】

import numpy as np
emb_dim = word_emb_model.vector_size    #要训练的数据集的向量大小
emb_table = []
for i, word in enumerate(word_list):  #['[PAD]', '[UNKNOWN]', 'do', 'hate', 'i', 'it', 'like', 'love', 'not', 'that']
  if word in word_emb_model:
    emb_table.append(word_emb_model[word])
  else:
    emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)  #16283

# Download Embeddings:

In [ ]:
#https://github.com/RaRe-Technologies/gensim-data
#下载 预训练 嵌入 pre_trained embedding
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-25") # corpus = api.load('text8')
#还有只下载路径不打开的。
'''
import gensim.downloader as api
print(api.load("20-newsgroups", return_path=True))  # output: /home/user/gensim-data/20-newsgroups/20-newsgroups.gz
print(api.load("glove-twitter-25", return_path=True))  # output: /home/user/gensim-data/glove-twitter-25/glove-twitter-25.gz
'''


In [ ]:
word_emb_model

In [ ]:
word_emb_model.most_similar("apple") #和它相似单词的词向量。

In [ ]:
#例如，load 一个语料库，+ 去训练word2vec model
tran_model = Word2Vec(word_emb_model) #NameError: name 'Word2Vec' is not defined
tran_model.most_similar("car")

#build word embeddings model

**Label encodding[标签编码]**

In [ ]:
import numpy as np

from sklearn.preprocessing import LabelEncoder

unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
# Please encode the labels (Do NOT add new lines of code in this section)
# Hint: Try to understand the difference between fit_transform and transform
#To normalise the data (so that the variance of the feature data is 1 and the mean is 0), the mean μ and variance σ^2 of the feature data are calculated. fit is not used because the training set (fit) has already found the mean and variance. (The transformation rule is found), and this rule is used directly.
label_train_encoded = lEnc.fit_transform(train_label)
label_test_encoded = lEnc.transform(test_label)

n_class = len(unique_labels)
print(unique_labels)
print(lEnc.transform(unique_labels))


**get word list**

In [ ]:
word_set = set() 
for sent in text_train_le:
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index)

**padding and encoding**

In [ ]:
# The sequence length is pre-defined, you can't change this value for this exercise
seq_length = 16

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 

In [ ]:
train_pad_encoded #[3259, 7339, 1161, 11990, 15077, 40, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72]

In [ ]:
#下载embedding
'''
import gensim.downloader as api
word_emb_model = api.load("xxx") # Download an embedding other than glove-twitter-25
'''

In [ ]:
#创建嵌入查询表  embedding lookup table 【是传说中的W吗？】 【为什么要嵌入这个词表】
import numpy as np
emb_dim = word_emb_model.vector_size    #要训练的数据集的向量大小
emb_table = []
for i, word in enumerate(word_list):  #['[PAD]', '[UNKNOWN]', 'do', 'hate', 'i', 'it', 'like', 'love', 'not', 'that']
  if word in word_emb_model:
    emb_table.append(word_emb_model[word])
  else:
    emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)

#WordEmbedding

In [ ]:
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')  



In [ ]:
# 下载Twitter data
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

In [ ]:
#look like
twitter_samples.fileids()
print("Negative LENGTH : " +  str(len(twitter_samples.strings('negative_tweets.json'))) )
print("Positive LENGTH : " +  str(len(twitter_samples.strings('positive_tweets.json'))) )
twitter_samples.strings('positive_tweets.json')[:2]
twitter_samples.strings('negative_tweets.json')[:2]

In [ ]:
#putting labels
label = "neg"
neg_dataset = [(label, instance) for instance in twitter_samples.strings('negative_tweets.json')]

label = "pos"
pos_dataset = [(label, instance) for instance in twitter_samples.strings('positive_tweets.json')]

print(neg_dataset[:2])
print(pos_dataset[:2])

In [ ]:
#splitting dataset 分隔

training_data = neg_dataset[:4000] + pos_dataset[:4000]
testing_data = neg_dataset[4000:] + pos_dataset[4000:]


####### relatively small dataset, can be used at development phase #######
dev_training_data = neg_dataset[:100] + pos_dataset[:100]
dev_testing_data = neg_dataset[100:200] + pos_dataset[100:200]


print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data)))
print("Size of testing dataset: {0}".format(len(testing_data)))
print("------------------------------------")
print("Size of dev training dataset: {0}".format(len(dev_training_data)))
print("Size of dev testing dataset: {0}".format(len(dev_testing_data)))
print("------------------------------------")

In [ ]:
#saving dataset into pickle files
import pickle
# write to cPickle
pickle.dump(training_data, open( "training_data.pkl", "wb" ) )
pickle.dump(testing_data, open( "testing_data.pkl", "wb" ) )

####### relatively small dataset, can be used at development phase #######
pickle.dump(dev_training_data, open( "dev_training_data.pkl", "wb" ) )
pickle.dump(dev_testing_data, open( "dev_testing_data.pkl", "wb" ) )

In [ ]:
#loading dataset from pickel
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '11CfwnSn-nN66U7YtCfCiJxSh3neZmKvk'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dev_training_data.pkl')  

id = '1AjVLvsK2p25eRsWWCyIqgP7fDs78UrIv'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dev_testing_data.pkl')  

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  


import pickle

dev_training_data = pickle.load(open("dev_training_data.pkl","rb"))
dev_testing_data = pickle.load(open("dev_testing_data.pkl","rb"))

training_data = pickle.load(open("training_data.pkl","rb"))
testing_data = pickle.load(open("testing_data.pkl","rb"))

In [ ]:
'''
# word embedding  2分 [lab2]
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model

      [从lab2,4,5随便选参数]embeddings_dimension ; learning_rate epochs;
  [lab5 有模型构建]
'''

'''
先变成词向量。
1变成向量矩阵。
句子长度不同。要做padding
np.mean()求均值。

模型： lab3
RNN
0-1 loss（这个好）或 交叉熵
linear： 叫分类器。 输出俩是p还是n
batch-size； hiddenstates层数；维度

'''
